In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import missingno as msno
import pandas_profiling
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#解决中文显示问题
plt.rcParams["font.sans-serif"] = [u"SimHei"]
plt.rcParams["axes.unicode_minus"] = False
#解决图中负号问题
plt.rcParams['axes.unicode_minus']=False

In [2]:
column_name=['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg','os','battery_capacity','storage','screen_size']

# names = ['id','target','card_type','netlong','owner_city','arpu','out_prdct_fee','prdct_fee','point_fee','myth_fee','brand','listing_date','listing_price','nettime','user_level','sex','family_client','group_client','music','expirationtime','video_member','other_tv','finance','bank','campus_client','migu_tv','age','adis','arpu_avg','myth_fee_avg','out_prdct_fee_avg','prdct_fee_avg']
original = pd.read_table('ZZ0702GuangFa_Samples100000_FeatureData.txt',names=column_name)
display(original.head(),original.shape)

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,13013302938,1,0.0,NaN,联通联通,9.0,0.0,0.0,0.0,0.0,...,NaN,NaN,38.863332,0.0,NaN,0.0,4.0,4120mAh,64GB,6.0
1,13022062073,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13035901978,1,NaN,NaN,NaN,37.2,0.0,0.0,0.0,0.0,...,NaN,NaN,37.013334,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,13042151031,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3050mAh,16/64GB,5.0
4,13045173577,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(103513, 36)

In [3]:
display(original.target.value_counts())

0    92989
1    10524
Name: target, dtype: int64

In [31]:
sample_false=original[original['target']==0].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
sample_true=original[original['target']==1].sample(n=10000, replace=False,random_state=0, axis=None).reset_index(drop=True)
combin_df=sample_true.append(sample_false)
combin_df.to_excel('d:\sample_model.xlsx',index=None)

In [35]:
sample_true.head()

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
0,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.48,29.0,NaN,NaN,NaN,NaN
1,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.0,4.0,2920mAH,16GB/32GB,5.0
2,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.20,135.0,4.0,3000mAh,32GB,5.5
3,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.00,90.0,4.0,3000mAh,16GB,5.7
4,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.0,3.0,1715mAh,16GB/64GB/128GB,4.7


In [36]:
# import random
def create_column_randomsample(column_random_dict):
    total_column_dict={}
    total_column_df=[]
    for columns_name in column_random_dict:
        if columns_name not in total_column_dict:
            total_column_dict[columns_name]=[]        
        for value_keys in column_random_dict[columns_name]:
            value_keys_counts=int(column_random_dict[columns_name][value_keys]*10000)
            for index in range(value_keys_counts):
                total_column_dict[columns_name].append(value_keys)
        #if columns_name=='婚姻状况':
            #print(columns_name,total_column_dict[columns_name])
        #print(pd.DataFrame(data=total_column_dict[columns_name],columns=[column_name]).sample(frac=1).reset_index(drop=True))
        #total_column_df.append(pd.DataFrame(data=total_column_dict[columns_name],columns=[column_name]).sample(frac=1).reset_index(drop=True))        
    
    result_df=pd.DataFrame(total_column_dict)
    for columns in result_df.columns:
        result_df[columns]=result_df[columns].sample(frac=1).reset_index(drop=True)
    #print(total_column_dict['婚姻状况'])
    #return pd.concat(total_column_df,axis=1)        
    #print(total_column_dict)
    #return pd.DataFrame(data=total_column_dict,columns=[column_name]).sample(frac=1)
    #print(result_df.tail(100))
    return result_df

def create_number_randomsample(number_random_dict):
    import numpy as np
    
    total_column_dict={}
    total_column_df=[]
    for columns_name in number_random_dict:
        #print(columns_name)
        if columns_name not in total_column_dict:
            #print(number_random_dict)
            number_column_value=np.random.normal(loc=number_random_dict[columns_name]['mean'],scale=number_random_dict[columns_name]['std'],size=10000)
            total_column_dict[columns_name]=number_column_value.astype(np.int32)
    return pd.DataFrame(total_column_dict)
            
            
True_column_random_dict={'性别':{'男':0.6,'女':0.4},'婚姻状况':{'已婚':0.65,'未婚':0.35},
                         '学历':{'小学':0.01,'初中':0.02,'高中':0.1,'初中中专':0.02,'高中中专':0.1,'大专':0.5,'本科':0.2,'研究生':0.05},
#                          '学位':{'其他':0.599,'学士':0.3,'硕士':0.1,'博士':0.001},
                         '征信认定年收入':{'5万':0.35,'10万':0.5,'20万':0.05,'30万':0.04,'40万':0.03,'50万':0.02,'大于50万':0.01},
                         '社会保障':{'1':0.7,'0':0.3},
                         '客户级信用额度':{'5000':0.05,'10000':0.15,'20000':0.3,'30000':0.3,'50000':0.2},
                         '美元账户额度':{'大于0':0.05,'小于0':0.95},
                         'CRM评级结果':{'普通':0.5,'准核心':0.3,'核心':0.2},
                         '客户评级结果':{'C1':0.15,'C2':0.15,'C3':0.3,'C4':0.2,'C5':0.15,'C6':0.05},
                         '借记卡层级':{'普卡':0.4,'金卡':0.3,'白金':0.25,'黑金':0.05},
                         '附卡卡数':{'0':0.6,'1':0.15,'2':0.1,'3':0.05,'4':0.05,'其他':0.05},
                         '存款期限':{'3个月':0.15,'6个月':0.15,'1年':0.3,'2年':0.2,'3年':0.15,'5年':0.05},
                         '是否为低保户':{'是':0.01,'否':0.99},
                         '受否贫困户':{'是':0.02,'否':0.98},
                        }
True_number_random_dict={'年龄':{'mean':30,'std':13.5},
                         '贷款放款本金':{'mean':500000,'std':150000},'账户余额':{'mean':200000,'std':100000.5},
                         '贷款期数':{'mean':120,'std':80},'剩余贷款本金':{'mean':300000,'std':100000},
                        }
                         

False_column_random_dict={'性别':{'男':0.45,'女':0.55},'婚姻状况':{'已婚':0.4,'未婚':0.6},
                         '学历':{'小学':0.02,'初中':0.03,'高中':0.1,'初中中专':0.1,'高中中专':0.4,'大专':0.3,'本科':0.03,'研究生':0.02},
#                          '学位':{'其他':0.599,'学士':0.3,'硕士':0.1,'博士':0.001},
                         '征信认定年收入':{'5万':0.55,'10万':0.35,'20万':0.04,'30万':0.03,'40万':0.02,'50万':0.008,'大于50万':0.002},
                         '社会保障':{'1':0.65,'0':0.35},
                         '客户级信用额度':{'5000':0.1,'10000':0.4,'20000':0.2,'30000':0.2,'50000':0.1},
                         '美元账户额度':{'大于0':0.01,'小于0':0.99},
                         'CRM评级结果':{'普通':0.7,'准核心':0.2,'核心':0.1},
                         '客户评级结果':{'C1':0.2,'C2':0.45,'C3':0.15,'C4':0.15,'C5':0.03,'C6':0.02},
                         '借记卡层级':{'普卡':0.6,'金卡':0.2,'白金':0.15,'黑金':0.05},
                         '附卡卡数':{'0':0.75,'1':0.1,'2':0.05,'3':0.05,'4':0.04,'其他':0.01},
                         '存款期限':{'3个月':0.1,'6个月':0.1,'1年':0.2,'2年':0.3,'3年':0.2,'5年':0.1},
                         '是否为低保户':{'是':0.05,'否':0.95},
                         '受否贫困户':{'是':0.07,'否':0.93},
                        }

False_number_random_dict={'年龄':{'mean':23,'std':13.5},
                         '贷款放款本金':{'mean':200000,'std':100000},'账户余额':{'mean':80000,'std':50000.5},
                         '贷款期数':{'mean':80,'std':60},'剩余贷款本金':{'mean':200000,'std':80000},
                        }


true_result_df=create_column_randomsample(True_column_random_dict)
true_result_number_df=create_number_randomsample(True_number_random_dict)

True_combin_df=true_result_df.merge(true_result_number_df,left_index=True, right_index=True)
True_combin_df=True_combin_df.merge(pd.DataFrame(sample_true[['id','owner_city']]),left_index=True, right_index=True)
True_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
True_combin_df['target']=1

False_result_df=create_column_randomsample(False_column_random_dict)
False_result_number_df=create_number_randomsample(False_number_random_dict)
False_combin_df=False_result_df.merge(False_result_number_df,left_index=True, right_index=True)
False_combin_df=False_combin_df.merge(pd.DataFrame(sample_false[['id','owner_city']]),left_index=True, right_index=True)
False_combin_df.rename(columns={'owner_city':'籍贯'},inplace=True)
False_combin_df['target']=0

total_comple=True_combin_df.append(False_combin_df).reset_index(drop=True)

In [37]:
display(True_combin_df.head(),False_combin_df.head(),total_comple.tail())
print(total_comple.shape,True_combin_df.shape,False_combin_df.shape)

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯,target
0,男,未婚,大专,5万,1,30000,大于0,核心,C2,黑金,...,否,否,34,655786,187498,94,159918,18746191488,黑龙江哈尔滨,1
1,女,已婚,本科,10万,0,20000,小于0,准核心,C1,黑金,...,否,否,32,341026,154197,166,317264,13769713610,云南曲靖,1
2,女,未婚,大专,10万,1,20000,小于0,普通,C4,金卡,...,否,否,28,648266,240635,95,490022,13838040676,河南郑州,1
3,男,已婚,大专,5万,0,30000,小于0,准核心,C3,普卡,...,否,否,49,568169,85009,1,178772,15945389676,黑龙江牡丹江,1
4,男,未婚,大专,5万,1,20000,小于0,普通,C3,金卡,...,否,否,38,442428,328406,49,203081,13888246777,云南昆明,1


,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯,target
0,男,未婚,高中中专,10万,1,10000,小于0,普通,C1,普卡,...,否,否,17,122064,101420,93,150242,17888631205,广东湛江,0
1,女,未婚,小学,5万,0,10000,小于0,准核心,C2,金卡,...,否,否,28,319290,-4609,29,221835,15864549598,山东济宁,0
2,女,未婚,初中中专,5万,1,30000,小于0,普通,C3,普卡,...,否,否,29,207860,126660,26,199313,13962340363,江苏苏州,0
3,女,未婚,高中中专,10万,1,10000,小于0,普通,C5,金卡,...,否,否,32,208242,65276,153,229508,18317249036,河南南阳,0
4,女,未婚,大专,50万,0,20000,小于0,普通,C2,普卡,...,否,否,8,228066,54612,180,283113,15706878054,浙江宁波,0


,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯,target
19995,女,未婚,高中,40万,0,5000,小于0,普通,C1,金卡,...,否,否,10,223355,4908,-7,331734,13640148563,广东韶关,0
19996,女,已婚,高中,5万,1,5000,小于0,普通,C2,黑金,...,是,否,27,175244,112309,17,138530,15106005956,福建泉州,0
19997,女,未婚,高中中专,20万,1,10000,小于0,核心,C3,金卡,...,否,否,-2,261950,72757,75,217833,15846369966,黑龙江哈尔滨,0
19998,女,已婚,高中中专,10万,0,5000,小于0,普通,C2,普卡,...,否,否,24,158032,212520,52,216150,15130057192,河北石家庄,0
19999,男,已婚,大专,5万,0,10000,小于0,普通,C4,金卡,...,否,否,20,150957,41856,1,135838,13565602962,新疆昌吉,0


(20000, 22) (10000, 22) (10000, 22)


In [38]:
total_comple.to_excel('d:/上海兴业银行样本数据集.xlsx',index=None)


In [8]:
True_combin_df.head()

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,...,存款期限,是否为低保户,受否贫困户,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金,id,籍贯
0,女,未婚,本科,5万,1,20000,小于0,核心,C4,普卡,...,6个月,否,否,10,539739,-7114,123,404910,13013302938,联通联通
1,男,已婚,大专,20万,1,20000,大于0,核心,C4,金卡,...,1年,否,否,48,807197,74019,63,404393,13022062073,NaN
2,男,已婚,高中中专,5万,1,10000,小于0,核心,C6,白金,...,1年,否,否,11,679981,216705,117,63766,13035901978,NaN
3,男,已婚,高中,5万,1,50000,小于0,核心,C1,白金,...,5年,否,否,30,498424,185155,201,330451,13042151031,NaN
4,男,已婚,大专,10万,1,50000,小于0,核心,C1,普卡,...,5年,否,否,12,511549,115947,-6,326409,13045173577,NaN


In [21]:
combin_df[['id','性别','籍贯','婚姻状况','学历','征信认定年收入','社会保障','客户级信用额度','美元账户额度','CRM评级结果','客户评级结果','借记卡层级','账户余额','剩余贷款本金']].to_excel('d:\上海兴业银行样本造数.xlsx',index=None)

In [164]:
display(true_result_df.head(),true_result_number_df.head())

,性别,婚姻状况,学历,征信认定年收入,社会保障,客户级信用额度,美元账户额度,CRM评级结果,客户评级结果,借记卡层级,附卡卡数,存款期限,是否为低保户,受否贫困户
0,男,已婚,大专,5万,1,30000,小于0,普通,C3,黑金,4,1年,否,否
1,男,已婚,高中,10万,1,30000,小于0,准核心,C3,普卡,0,3个月,否,否
2,女,已婚,本科,5万,0,20000,小于0,普通,C4,普卡,3,1年,否,否
3,男,已婚,大专,10万,1,20000,小于0,普通,C3,金卡,1,2年,否,否
4,女,已婚,小学,5万,1,20000,小于0,普通,C3,白金,0,5年,否,否


,年龄,贷款放款本金,账户余额,贷款期数,剩余贷款本金
0,37,691320,8,317,445082
1,15,645061,29,49,196353
2,19,579867,29,157,368844
3,10,667989,34,33,329207
4,30,566405,51,118,330431


In [12]:
pd.DataFrame

,id,target,card_type,netlong,owner_city,arpu,out_prdct_fee,prdct_fee,point_fee,myth_fee,...,age,adis,arpu_avg,myth_fee_avg,out_prdct_fee_avg,prdct_fee_avg,os,battery_capacity,storage,screen_size
97032,18746191488,1,1.0,37.0,黑龙江哈尔滨,45.32,6.48,29.0,0.0,0.0,...,NaN,1.0,45.320000,0.0,6.480000,29.000000,NaN,NaN,NaN,NaN
29750,13769713610,1,0.0,NaN,云南曲靖,115.70,0.00,0.0,0.0,0.0,...,NaN,NaN,115.099998,0.0,NaN,0.000000,4.0,2920mAH,16GB/32GB,5.0
36394,13838040676,1,1.0,NaN,河南郑州,143.20,0.20,135.0,9951.0,0.0,...,NaN,1.0,148.810003,0.0,0.200000,135.000000,4.0,3000mAh,32GB,5.5
79629,15945389676,1,0.0,30.0,黑龙江牡丹江,60.99,1.00,90.0,0.0,0.0,...,NaN,NaN,60.990002,0.0,1.000000,90.000000,4.0,3000mAh,16GB,5.7
40228,13888246777,1,1.0,NaN,云南昆明,258.00,0.00,0.0,0.0,0.0,...,NaN,NaN,258.000000,0.0,NaN,0.000000,3.0,1715mAh,16GB/64GB/128GB,4.7
82230,15980726225,1,1.0,NaN,福建福州,78.00,0.00,62.0,0.0,0.0,...,35.0,NaN,91.333333,0.0,NaN,75.333333,4.0,2300mAh,16GB,5.0
62550,15217163366,1,2.0,50.0,广东珠海,55.12,0.00,28.0,0.0,0.0,...,NaN,NaN,54.293334,0.0,NaN,28.000000,4.0,3100mAh,8GB,5.5
99664,18811186422,1,1.0,43.0,北京北京,118.00,0.00,118.0,0.0,0.0,...,NaN,NaN,118.224998,0.0,NaN,118.000000,4.0,3000mAh,16GB,5.5
93368,18374069022,1,0.0,44.0,湖南株洲,31.49,0.00,0.0,0.0,0.0,...,NaN,NaN,22.496667,0.0,NaN,0.000000,NaN,NaN,NaN,NaN
13231,13580829297,1,0.0,NaN,广东东莞,178.00,0.00,173.0,0.0,0.0,...,NaN,NaN,176.866669,0.0,NaN,171.333333,4.0,3750mAh,64GB/128GB/256GB,5.5
